In [2]:
import numpy as np

In [3]:
img = np.arange(16).reshape((4, 4)) + 1
img

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [4]:
def box_sum(img: np.ndarray, k: int) -> tuple[np.ndarray, np.ndarray]:
    h, w = img.shape
    size_y = h - k + 1
    size_x = w - k + 1
    hv_padded = np.pad(img, ((1, 0), (1, 0)), mode="constant")
    sums = np.zeros((size_y, size_x), dtype=np.int32)

    for y in range(1, hv_padded.shape[0]):
        running_sum = 0

        for x in range(1, hv_padded.shape[1]):
            # subtract the diagonal element to prevent double-counting
            running_sum += hv_padded[y, x] + hv_padded[y - 1, x] - hv_padded[y - 1, x - 1]
            hv_padded[y, x] = running_sum

            if y >= k and x >= k:
                box_sum = hv_padded[y, x] - hv_padded[y, x - k] - hv_padded[y - k, x] + hv_padded[y - k, x - k]
                sums[y - k, x - k] = box_sum

    return hv_padded, sums

In [9]:
k = 3

ii, sum = box_sum(img, k)

In [10]:
ii

array([[  0,   0,   0,   0,   0],
       [  0,   1,   3,   6,  10],
       [  0,   6,  14,  24,  36],
       [  0,  15,  33,  54,  78],
       [  0,  28,  60,  96, 136]])

In [11]:
sum

array([[54, 63],
       [90, 99]], dtype=int32)

In [8]:
from numpy.lib.stride_tricks import sliding_window_view


def _ref_box_sum(img: np.ndarray, k: int) -> np.ndarray:
    w = sliding_window_view(img, (k, k))

    return w.astype(np.float64).sum(axis=(2, 3))

def run_box_sum_tests(box_sum_fn):
    rng = np.random.default_rng(42)
    cases = [
        (rng.integers(0, 10,  (3, 3),  dtype=np.int32),  [2]),
        (rng.integers(0, 255, (5, 7),  dtype=np.uint8),  [2, 3]),
        (rng.integers(-50, 50, (16, 10), dtype=np.int32), [3, 4]),
        (rng.integers(0, 1000,(12, 12), dtype=np.int64), [6]),
    ]

    for idx, (img, ks) in enumerate(cases, 1):
        for k in ks:
            ref = _ref_box_sum(img, k)
            _, out = box_sum_fn(img, k)
            np.testing.assert_allclose(
                np.asarray(out, dtype=np.float64),
                np.asarray(ref, dtype=np.float64),
                rtol=0, atol=0
            )
    print("All box_sum tests passed on 5 matrices and multiple window sizes.")


run_box_sum_tests(box_sum)

/tmp/ipykernel_97658/588365313.py:13: RuntimeWarning: overflow encountered in scalar add
  running_sum += hv_padded[y, x] + hv_padded[y - 1, x] - hv_padded[y - 1, x - 1]
/tmp/ipykernel_97658/588365313.py:13: RuntimeWarning: overflow encountered in scalar subtract
  running_sum += hv_padded[y, x] + hv_padded[y - 1, x] - hv_padded[y - 1, x - 1]
/tmp/ipykernel_97658/588365313.py:17: RuntimeWarning: overflow encountered in scalar subtract
  box_sum = hv_padded[y, x] - hv_padded[y, x - k] - hv_padded[y - k, x] + hv_padded[y - k, x - k]
/tmp/ipykernel_97658/588365313.py:17: RuntimeWarning: overflow encountered in scalar add
  box_sum = hv_padded[y, x] - hv_padded[y, x - k] - hv_padded[y - k, x] + hv_padded[y - k, x - k]


AssertionError: 
Not equal to tolerance rtol=0, atol=0

Mismatched elements: 23 / 24 (95.8%)
Max absolute difference among violations: 768.
Max relative difference among violations: 0.9974026
 ACTUAL: array([[ 37., 104.,  84.,  91.,  69., 203.],
       [133.,  54.,  92., 158.,   2., 179.],
       [ 85.,  74., 146.,   8.,  85.,   2.],
       [ 35., 146.,  59., 186., 221., 218.]])
 DESIRED: array([[549., 360., 340., 603., 581., 459.],
       [645., 566., 348., 670., 770., 435.],
       [597., 586., 402., 520., 597., 514.],
       [547., 658., 571., 442., 221., 474.]])